# Other Code

Given a fits image of a survey, calculate the area of the survey in degrees^2. The fits image is in pixels and the CDELT1 value on the header is used to rescale to physical units.

In [ ]:
def calculate_survey_area(fits_file):
    # Open the FITS file and extract the header
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        data = hdul[0].data
        
        # Ensure data is at least 2D (handle multi-dimensional FITS files)
        if data.ndim > 2:
            data = data[0]  # Take the first frame if it's a multi-frame FITS
        
        # Get image dimensions
        height, width = data.shape[-2:]  # Ensure correct shape indexing
        total_pixels = width * height
        
        # Extract pixel scale from header (if available)
        if 'PIXSCALE' in header:
            pixel_scale = header['PIXSCALE']  # in arcsec/pixel
        elif 'CDELT1' in header and 'CDELT2' in header:
            if header['CUNIT1'] == 'deg' and header['CUNIT2'] == 'deg': # Check if units are in degrees
                pixel_scale = np.abs(header['CDELT1']) * 3600  # Convert from decimal degrees to arcseconds
        else:
            raise ValueError("Pixel scale not found in FITS header. Check metadata or specify manually.")
        
        # Compute the survey area in square arcseconds
        area_arcsec2 = total_pixels * (pixel_scale ** 2)
        
        # Convert to square degrees
        area_deg2 = area_arcsec2 / (3600 ** 2)
        
    print(f"Total pixels: {total_pixels}")
    print(f"Pixel scale (arcsec/pixel): {pixel_scale}")
    print(f"Survey area (deg^2): {area_deg2}")

    return area_deg2

In [ ]:
area_emu_survey = calculate_survey_area(emu_mosaic_path)